In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
import tensorflow as tf

# 构建一个多层卷积网络

### 权重初始化

tf.truncated_normal()：函数返回具有来自正态分布的随机值的张量，该正态分布的幅度与平均值相差不超过2个标准偏差。

In [11]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### 卷积和池化

[卷积 : ∫f(t)g(u-t)dt](https://www.zhihu.com/question/22298352) 

[卷积特征提取](http://ufldl.stanford.edu/wiki/index.php/%E5%8D%B7%E7%A7%AF%E7%89%B9%E5%BE%81%E6%8F%90%E5%8F%96)

[tf.nn.conv2d是TensorFlow里面实现卷积的函数](http://www.cnblogs.com/welhzh/p/6607581.html)

[池化](http://ufldl.stanford.edu/wiki/index.php/%E6%B1%A0%E5%8C%96)

[max pooling是CNN当中的最大值池化操作](https://www.cnblogs.com/qggg/p/6832705.html)

CNN(卷积神经网络)、RNN(循环神经网络)、DNN(深度神经网络)

In [12]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

### 第一层卷积

In [13]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [15]:
x = tf.placeholder("float", shape=[None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

tf.nn.relu(features, name = None)

解释：这个函数的作用是计算激活函数relu，即max(features, 0)。即将矩阵中每行的非最大值置0。

In [16]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### 第二层卷积

In [17]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### 密集连接层

[全连接层](https://www.zhihu.com/question/41037974)（fully connected layers，FC）

在整个卷积神经网络中起到“分类器”的作用。

如果说卷积层、池化层和激活函数层等操作是将原始数据映射到隐层特征空间的话，全连接层则起到将学到的“分布式特征表示”映射到样本标记空间的作用。

In [18]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

为了减少过拟合，我们在输出层之前加入dropout。

tf.nn.dropout操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出值的scale。

In [19]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 输出层

In [20]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 训练和评估模型

In [22]:
y_ = tf.placeholder("float", [None,10])

In [32]:
sess = tf.InteractiveSession()

In [33]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess.run(tf.initialize_all_variables())

In [36]:
for i in range(1000):
  batch = mnist.train.next_batch(100)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print ("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print ("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.99
step 100, training accuracy 1
step 200, training accuracy 1
step 300, training accuracy 0.99
step 400, training accuracy 1
step 500, training accuracy 1
step 600, training accuracy 0.99
step 700, training accuracy 0.99
step 800, training accuracy 1
step 900, training accuracy 0.99
test accuracy 0.9914
